<a href="https://colab.research.google.com/github/RakshiSree/CodeAlpha_Task/blob/main/MusicGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install music21 tensorflow


In [9]:
from google.colab import files
uploaded = files.upload()  # Upload a single .mid file (e.g., a piano piece)


Saving Queen - Bohemian Rhapsody.mid to Queen - Bohemian Rhapsody.mid


In [10]:
from music21 import converter, instrument, note, chord

notes = []

file = list(uploaded.keys())[0]  # Get uploaded filename

# Ensure it's really a .mid file
if not file.endswith('.mid'):
    raise ValueError("❌ Please upload a .mid MIDI file, not .mp3 or audio formats!")

midi = converter.parse(file)
parts = instrument.partitionByInstrument(midi)
elements = parts.parts[0].recurse() if parts else midi.flat.notes

for el in elements:
    if isinstance(el, note.Note):
        notes.append(str(el.pitch))
    elif isinstance(el, chord.Chord):
        notes.append('.'.join(str(n) for n in el.normalOrder))


In [11]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation
from tensorflow.keras.utils import to_categorical

# Prepare sequences
note_names = sorted(set(notes))
note_to_int = {note: i for i, note in enumerate(note_names)}
int_to_note = {i: note for note, i in note_to_int.items()}

sequence_length = 50
X, y = [], []

for i in range(len(notes) - sequence_length):
    seq_in = [note_to_int[n] for n in notes[i:i + sequence_length]]
    seq_out = note_to_int[notes[i + sequence_length]]
    X.append(seq_in)
    y.append(seq_out)

n_vocab = len(note_names)
X = np.reshape(X, (len(X), sequence_length, 1)) / float(n_vocab)
y = to_categorical(y)

# Build LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], 1), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(128))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=20, batch_size=64)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 411ms/step - loss: 3.1217
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 437ms/step - loss: 2.9662
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 438ms/step - loss: 2.8770
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 459ms/step - loss: 2.7546
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 398ms/step - loss: 2.7727
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 408ms/step - loss: 2.6740
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 397ms/step - loss: 2.6747
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 637ms/step - loss: 2.6942
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 699ms/step - loss: 2.7127
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 394ms/step - loss: 2.6864
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 379ms/step - loss: 2.6977
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 400ms/step - loss: 2.7085
Epoch 13/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 410ms/step - loss: 2.6543
Epoch 14/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 385ms/step - loss: 2.7208
Epoch 15/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 385ms/step - loss: 2.7064
Epoch 16/20
2/2 ━━━

In [14]:
generated_notes = []
pattern = X[start].copy()  # shape: (50, 1)

for _ in range(100):
    prediction = model.predict(pattern.reshape(1, sequence_length, 1), verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    generated_notes.append(result)

    # Correctly reshape and append
    next_input = np.array([[index / float(n_vocab)]])
    pattern = np.vstack((pattern[1:], next_input))


In [15]:
# Convert generated notes to MIDI
from music21 import stream, note, chord

output_notes = []
for pattern in generated_notes:
    if '.' in pattern:
        chord_notes = [int(n) for n in pattern.split('.')]
        new_chord = chord.Chord(chord_notes)
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        output_notes.append(new_note)

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='generated_music.mid')

# Download
from google.colab import files
files.download('generated_music.mid')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>